# Load Variables

In [1]:
%run "scripts/load_daily_variables.py"

Loading Variables: 100%|██████████| 26/26 [01:10<00:00,  2.72s/it]


# High Beta with Recent Elevated Volatility and Activity

In [2]:
hbeta_vol_regime_5day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any()
                    ]
hbeta_vol_regime_20day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (sym not in hbeta_vol_regime_5day)
                    ]

- Quant Rating of stocks with elevated volatility in the last 5 days

In [3]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_5day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
LASR    4.981132
GRAL    4.960692
W       4.927673
CRDO    4.919811
CYH     4.872642
FLNC    4.847484
EBS     4.753145
GLXY    4.633648
BFLY    4.480170
QUBT    4.199717
BBAI    4.145892
DAN     4.066572
INGM    3.950425
PACB    3.899433
AMRC    3.797450
VIK     3.729462
RIOT    3.689802
ONDS    3.494915
IREN    3.486441
LMND    3.484068
CIFR    3.466441
AMPX    3.440339
ASTS    3.434576
CTMX    3.434237
LC      3.426441
HOOD    3.389492
RDDT    3.386780
RKT     3.385085
NVAX    3.231525
DPRO    3.229831
UMAC    3.182712
SPIR    3.091864
AEHR    3.066780
QS      3.064407
U       3.057288
NCLH    3.003729
RUN     2.998644
SHOP    2.984407
KOPN    2.916271
CABA    2.877288
RYAM    2.855254
AEVA    2.823051
ARM     2.803729
APPS    2.796271
NTLA    2.757288
TBCH    2.755932
CG      2.755254
RDW     2.715593
JANX    2.715254
NNE     2.690847
Name: 2026-01-22, dtype: float64

- Quant Rating of stocks with elevated volatility in the last 20 days minus those in hbeta_vol_regime_5day

In [4]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_20day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
TPC     4.844340
CTRN    4.823899
ADPT    4.704403
GAP     4.694969
APP     4.635220
PSIX    4.610063
AS      4.562893
DAVE    4.531447
CDNA    4.466006
NVTS    4.131728
INOD    3.876771
SFIX    3.865439
LSPD    3.686969
SKYT    3.499322
SLS     3.496949
BE      3.492881
PRAX    3.492203
COMP    3.490291
APLD    3.482373
REAL    3.479322
SMTC    3.425424
SOFI    3.423051
JMIA    3.415593
SEZL    3.394915
TMDX    3.370169
TXG     3.369153
ADEA    3.360339
CENX    3.352542
NBIS    3.337966
NXT     3.321017
OMER    3.265085
BTBT    3.235254
Z       3.218447
AFRM    3.157966
AMSC    3.135593
ACHR    3.133559
BKSY    3.108814
SAIA    3.097627
NAGE    3.096949
AUR     3.053898
FULC    3.029153
PTGX    2.981017
QXO     2.977966
PTON    2.975932
OPEN    2.966019
TREE    2.961356
ODD     2.953559
WRBY    2.949831
ULCC    2.932542
INMD    2.883390
Name: 2026-01-22, dtype: float64

# YTD Performance vs. Quant Rating

In [5]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = daily_quant_rating_df[quant_col].reset_index().rename(columns={quant_col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')
# df = df.loc[df.Ticker.isin(fu.sector_industry_member_search('Basic Materials', level='sector'))]
# Assign color based on logic:
# quant rating > 4.8 & Performance < 50 -> green
# quant rating < 3 & Performance > 50 -> red
# else -> gray
def assign_color(row):
    if (row['Quant Rating'] > 4.8) and (row['Performance (YearToDate)'] < 50):
        return 'green'
    elif (row['Quant Rating'] < 3) and (row['Performance (YearToDate)'] > 50):
        return 'red'
    elif (row['Quant Rating'] > 2.5) and (row['Quant Rating'] < 4) and (row['Performance (YearToDate)'] > 50):
        return 'yellow'
    else:
        return 'gray'

df['Color'] = df.apply(assign_color, axis=1)

px.scatter(
    df.assign(
        Color_Label=df['Color'].map({'gray': 'uncertain', 'green': 'undervalued', 'red': 'overvalued', 'yellow': 'worth watching'})
    ),
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    color='Color_Label',
    color_discrete_map={'uncertain': 'gray', 'undervalued':'green', 'overvalued':'red', 'worth_watching': 'yellow'},
    category_orders={'Color_Label': ['uncertain', 'undervalued', 'overvalued', 'worth_watching']},
    labels={'Color_Label': 'Category'},
    title='YTD Performance vs. Quant Rating'
)

# Interest List Long Performance vs. Quant Rating

- Quant Rating for symbol in interest_list_long

In [6]:
mask = daily_quant_rating_df.index.isin(interest_list_long)
col = daily_quant_rating_df.columns[-2]
quantdf_illong = (
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
)
quantdf_illong.head(50)

Symbol
MU      4.998428
NGD     4.990566
IAG     4.988994
TTMI    4.984277
LASR    4.981132
GOLD    4.971698
CIEN    4.968553
NEM     4.966981
GRAL    4.960692
GFI     4.957547
TSM     4.954403
CDE     4.951258
NUTX    4.949686
AMKR    4.944969
COHR    4.940252
INDV    4.938679
KGC     4.937107
UPB     4.935535
LQDA    4.933962
CSTM    4.932390
CGAU    4.930818
VSAT    4.929245
W       4.927673
GM      4.926101
AU      4.924528
AA      4.922956
HUT     4.921384
CRDO    4.919811
HBM     4.918239
NBR     4.916667
GH      4.915094
CLS     4.911950
DRH     4.906250
WLDN    4.900943
AEM     4.893082
CIB     4.885220
RERE    4.880503
PAAS    4.878931
EGO     4.877358
RIGL    4.871069
BVN     4.867925
KALU    4.853774
ERO     4.852201
LCII    4.849057
FLNC    4.847484
TSEM    4.838050
KRYS    4.831761
OI      4.827044
ENVA    4.822327
VIAV    4.817610
Name: 2026-01-22, dtype: float64

- Quant Rating vs. YTD Performance for symbols in interest_list_long

In [7]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = quantdf_illong.reset_index().rename(columns={col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')
# df = df.loc[df.Ticker.isin(fu.sector_industry_member_search('Basic Materials', level='sector'))]
# Assign color based on logic:
# quant rating > 4.8 & Performance < 50 -> green
# quant rating < 3 & Performance > 50 -> red
# else -> gray
def assign_color(row):
    if (row['Quant Rating'] > 4.8) and (row['Performance (YearToDate)'] < 50):
        return 'green'
    elif (row['Quant Rating'] < 3) and (row['Performance (YearToDate)'] > 50):
        return 'red'
    elif (row['Quant Rating'] > 2.5) and (row['Quant Rating'] < 4) and (row['Performance (YearToDate)'] > 50):
        return 'yellow'
    else:
        return 'gray'

df['Color'] = df.apply(assign_color, axis=1)

px.scatter(
    df.assign(
        Color_Label=df['Color'].map({'gray': 'uncertain', 'green': 'undervalued', 'red': 'overvalued', 'yellow': 'worth watching'})
    ),
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    color='Color_Label',
    color_discrete_map={'uncertain': 'gray', 'undervalued':'green', 'overvalued':'red', 'worth_watching': 'yellow'},
    category_orders={'Color_Label': ['uncertain', 'undervalued', 'overvalued', 'worth_watching']},
    labels={'Color_Label': 'Category'},
    title='YTD Performance vs. Quant Rating'
)

# INTEREST LIST LONG FROM FUNDAMENTALS

- Sorted by Quant Rating

In [8]:
interest_list_fundamental = [
    sym for sym in symbols if 
    (symbols[sym].interest_factor is not None) and
    ('Fundamental' in symbols[sym].interest_factor)
]
col = daily_quant_rating_df.columns[-2]
quantdf_ifund = (
daily_quant_rating_df.loc[daily_quant_rating_df.index.isin(interest_list_fundamental), col]
.sort_values(ascending=False)
)
quantdf_ifund.head(50)

Symbol
MU      4.998428
NGD     4.990566
IAG     4.988994
TTMI    4.984277
LASR    4.981132
GOLD    4.971698
CIEN    4.968553
NEM     4.966981
GRAL    4.960692
GFI     4.957547
TSM     4.954403
CDE     4.951258
NUTX    4.949686
AMKR    4.944969
COHR    4.940252
INDV    4.938679
KGC     4.937107
UPB     4.935535
LQDA    4.933962
CSTM    4.932390
CGAU    4.930818
VSAT    4.929245
W       4.927673
GM      4.926101
AU      4.924528
AA      4.922956
HUT     4.921384
CRDO    4.919811
HBM     4.918239
NBR     4.916667
GH      4.915094
CLS     4.911950
DRH     4.906250
WLDN    4.900943
Name: 2026-01-22, dtype: float64

- Sorted by Relative Volume

In [9]:
sorted(
    {sym: symbols[sym].RVol.iloc[-1].item() for sym in interest_list_fundamental}.items(),
    key=lambda x: x[1],
    reverse=True
)

[('CLS', 2.723210621608158),
 ('NGD', 2.049967920612772),
 ('GRAL', 1.8993849869353576),
 ('CDE', 1.6176463143395692),
 ('GH', 1.6020140717160216),
 ('DRH', 1.5662506117433734),
 ('NUTX', 1.5154026478062943),
 ('LASR', 1.495175795453222),
 ('AU', 1.4945902640394042),
 ('IAG', 1.3601700501405),
 ('CRDO', 1.3524035081808305),
 ('CSTM', 1.2840498360226271),
 ('KGC', 1.2317004632432849),
 ('MU', 1.220129726134232),
 ('TTMI', 1.2111937010619152),
 ('W', 1.2089056687901316),
 ('GFI', 1.2061391338246434),
 ('HBM', 1.1342563148925966),
 ('AA', 1.1099552324605568),
 ('AMKR', 1.1048502963297913),
 ('COHR', 1.0635110451066467),
 ('CGAU', 1.049545154751744),
 ('NEM', 1.0195082489120297),
 ('LQDA', 1.002773525932953),
 ('CIEN', 0.9482249520426458),
 ('GM', 0.9472014397579932),
 ('TSM', 0.9386300739889504),
 ('VSAT', 0.8911928326624751),
 ('GOLD', 0.7742075030811288),
 ('NBR', 0.7724639672520228),
 ('INDV', 0.7376343174225163),
 ('UPB', 0.7237504896038534),
 ('HUT', 0.705612013518363),
 ('WLDN', 0.5

- Quant Rating vs. YTD Performance for symbols with fundamental factor in interest_list_long

In [10]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = quantdf_ifund.reset_index().rename(columns={col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')

px.scatter(
    df,
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    title='YTD Performance vs. Quant Rating'
)